# Romo - A General Knowledge Bot- TFIDF

Load Required Library

In [1]:
#!pip install nltk
import pandas as pd
import os
import json
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk import pos_tag
import numpy as np
import pickle
import string
import random
import timeit

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer 

import warnings
warnings.simplefilter('ignore')

In [11]:

nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

In [13]:
nltk.download('universal_tagset')

[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\universal_tagset.zip.


True

In [15]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### Data Preparation - Convert CSV to JSON

- Load Data

In [4]:
convdata = pd.read_csv('general_knowledge.csv')

#show header of the dataset
convdata.head(10)

,question,answer
0,WHAT IS THE NAME OF THE HORSE-LIKE ANIMAL WITH...,ZEBRA
1,WHAT IS THE NAME OF THE LONG SLEEP SOME ANIMAL...,HIBERNATION
2,WHAT IS THE NAME OF THE RUBBER OBJECT THAT IS ...,PUCK
3,WHAT IS THE NAME OF THE REMAINS OF PLANTS AND ...,FOSSILS
4,WHICH PRECIOUS GEM IS RED?,RUBY
5,WHAT IS THE NAME OF THE SEVERE HEADACHE THAT R...,MIGRAINE
6,WHAT IS THE LAST NAME OF THE AUTHOR WHO WROTE ...,SHAKESPEARE
7,WHAT IS THE NAME OF A DRIED GRAPE?,RAISIN
8,WHAT IS THE NAME OF THE COMIC STRIP CHARACTER ...,POPEYE
9,WHAT ANIMAL RUNS THE FASTEST?,CHEETAH


In [5]:
# Lets see size of data
convdata.shape

(1993, 2)

Define some greethings and if any of these greetings are uttered, bot will reply one out the resoponses

In [6]:
#greeting function
GREETING_INPUTS = ("hello", "hi", "greetings", "hello i need help", "good day","hey","i need help", "greetings")
GREETING_RESPONSES = ["Good day, How may i of help?", "Hello, How can i help?", "hello", "I am glad! You are talking to me."]
           
def greeting(sentence):
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

In [ ]:
### Define Text preprocessing Functions

In [7]:
#Wordnet Lemmatization 

lemmer = nltk.WordNetLemmatizer()
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]

In [8]:
# Remove punctuation
def RemovePunction(tokens):
    return[t for t in tokens if t not in string.punctuation]

In [9]:
# Create a stopword list from the standard list of stopwords available in nltk
stop_words = set(stopwords.words('english'))
print(len(stop_words))

179


## Model Training

We will train model and store then model using pickle, model is trained so in future we can simply load and start using model.

For training simply preprocess text and convert questions to tfidf vectors and save vectors

In [16]:
tfidf_vectorizer_pickle_path = "tfidf_vectorizer.pkl"
tfidf_matrix_pickle_path = "tfidf_matrix_train.pkl"
start = timeit.default_timer()
i=0
sentences = []
        
for index, row in convdata.iterrows():
    db_tokens = RemovePunction(nltk.word_tokenize(row['question']))
    pos_db_tokens = [word for word,pos in pos_tag(db_tokens, tagset='universal')]
    db_word_tokens = LemTokens(pos_db_tokens)
                
    db_filtered_sentence = [] 
    for dbw in db_word_tokens: 
        if dbw not in stop_words: 
            db_filtered_sentence.append(dbw)  
                
    db_filtered_sentence =" ".join(db_filtered_sentence).lower()
                
    #Debugging Checkpoint
    #print('TRAINING INPUT: '+db_filtered_sentence)
                
    sentences.append(db_filtered_sentence)
    i +=1                
    # ---------------------------------------------------------------------------#
                
tfidf_vectorizer = TfidfVectorizer() 
tfidf_matrix_train = tfidf_vectorizer.fit_transform(sentences)
        
#train timing
stop = timeit.default_timer()
print ("Training Time : ")
print (stop - start) 
    
f = open(tfidf_vectorizer_pickle_path, 'wb')
pickle.dump(tfidf_vectorizer, f) 
f.close()
    
f = open(tfidf_matrix_pickle_path, 'wb')
pickle.dump(tfidf_matrix_train, f) 
f.close() 
# ------------------------------------------#


Training Time : 
16.9038768677147


In [17]:
# Tfidf vectors are numbers and if we want to see them, will see a lot of zeros 
print (type(tfidf_matrix_train))
a = tfidf_matrix_train.todense()
print(a)

<class 'scipy.sparse.csr.csr_matrix'>
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [18]:
def Talk_To_Romo(test_set_sentence):
    tfidf_vectorizer_pickle_path = "tfidf_vectorizer.pkl"
    tfidf_matrix_pickle_path = "tfidf_matrix_train.pkl"
    
    i = 0
    sentences = []
    
    # ---------------Tokenisation of user input -----------------------------#
    
    tokens = RemovePunction(nltk.word_tokenize(test_set_sentence))
    pos_tokens = [word for word,pos in pos_tag(tokens, tagset='universal')]
    
    word_tokens = LemTokens(pos_tokens)
    
    filtered_sentence = []
    for w in word_tokens: 
        if w not in stop_words: 
            filtered_sentence.append(w)  
    
    filtered_sentence =" ".join(filtered_sentence).lower()
            
    test_set = (filtered_sentence, "")
    f = open(tfidf_vectorizer_pickle_path, 'rb')
    tfidf_vectorizer = pickle.load(f)
    f.close()
        
    f = open(tfidf_matrix_pickle_path, 'rb')
    tfidf_matrix_train = pickle.load(f)
    
    #For Tracing, comment to remove from print.
    #print('USER INPUT:'+filtered_sentence)
    
    # -----------------------------------------------------------------------#
                
    #use the learnt dimension space to run TF-IDF on the query
    tfidf_matrix_test = tfidf_vectorizer.transform(test_set)

    #then run cosine similarity between the 2 tf-idfs
    cosine = cosine_similarity(tfidf_matrix_test, tfidf_matrix_train)
    
    #if not in the topic trained.no similarity 
    idx= cosine.argsort()[0][-2]
    flat =  cosine.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    
    if (req_tfidf==0):
        
        not_understood = "Apology, I do not understand. Can you rephrase?"
        
        return not_understood, not_understood, 2
        
    else:
        cosine = np.delete(cosine, 0)

        #get the max score
        max = cosine.max()
        response_index = 0

        #if max score is lower than < 0.34 > (we see can ask if need to rephrase.)
        if (max <= 0.34): #Threshold B
            
            not_understood = "Apology, I do not understand. Can you rephrase?"
            
            return not_understood,not_understood, 2
        else:
                response_index = np.where(cosine == max)[0][0] 
                j = 0 
                return convdata.iloc[response_index+1]['answer'], convdata.iloc[response_index+1]['question'], max
                with open(json_file_path, "r") as sentences_file:
                    reader = json.load(sentences_file)
                    for row in reader:
                        j += 1 
                        if j == response_index: 
                            return row["answer"], row["question"], max
                            break

In [20]:
flag=True
# Lets add some docorations around
print("......................................................................................")
print('\x1b[1;37;40m' + 'Romo'+'\x1b[0m'+': '+ 'My name is Romo, a General Knowledge Bot.')
print('\x1b[1;37;40m' + 'Romo'+'\x1b[0m'+': '+ 'I will try my best to answer your query.')
print('\x1b[1;37;40m' + 'Romo'+'\x1b[0m'+': '+ 'If you want to exit, you can type < bye >.')
while(flag==True):
    print("......................................................................................")
    sentence = input('\x1b[0;30;47m' +"USER  "+'\x1b[0m'+":")
    print("......................................................................................")
    if(sentence.lower()!='bye'):
        if(greeting(sentence.lower())!=None):
            print('\x1b[1;37;40m' + 'Romo'+'\x1b[0m'+': '+ greeting(sentence.lower()))
        else:
            response_primary, response_message, line_id_primary = Talk_To_Romo(sentence)
            print('\x1b[1;37;40m' + 'Romo'+'\x1b[0m'+': '+response_primary)
    else:
        flag=False
print('\x1b[1;37;40m' + 'Romo'+'\x1b[0m'+': '+"Bye! Hope that i am of help.") 

......................................................................................
Romo: My name is Romo, a General Knowledge Bot.
Romo: I will try my best to answer your query.
Romo: If you want to exit, you can type < bye >.
......................................................................................
USER  :which is the largest city of romania
......................................................................................
Romo: Bucharest
......................................................................................
USER  :which is the fastest animal
......................................................................................
Romo: CHEETAH
......................................................................................
USER  :bye
......................................................................................
Romo: Bye! Hope that i am of help.


## Questions you can ask to Robo:

- Which is fastest animal
- which is the largest city of Romania
- what are the most famous rocks are in Finland

To finish the prorgam: 
simply say bye